# Transfer-Learning-Pipeline

This pipeline Contains The Implementation of 12 Transfer Learning Model.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import pathlib
import sys

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


## Contents

### Directories

In [ ]:
# go to parent directory
os.chdir("../..")
import paths

REPO_DIR = paths.get_repo_path()
ROOT_DIR = REPO_DIR / "Stanford Dogs"
DATA_BASE_PATH = paths.get_data_path() / "stanford-dogs-dataset"
DATA_PATH = DATA_BASE_PATH / "splited-data"

TRAIN_PATH = DATA_PATH / "train"
TEST_PATH = DATA_PATH / "test"

# set path to repo_dir
os.chdir(REPO_DIR)

## Data

### Variables

In [ ]:
RANDOM_SEED = 42
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

VALIDATION_SPLIT = 0.2

BATCH_SIZE = 64

CLASS_NAMES = sorted([img_cls for img_cls in  os.listdir(TRAIN_PATH) if img_cls != ".DS_Store"])
NUM_CLASSES = len(CLASS_NAMES)

# MODEL
MODEL_PATH = ROOT_DIR / "models"
LOG_PATH = ROOT_DIR / "log"



In [ ]:
tf.random.set_seed(RANDOM_SEED)

### Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale=1./255,
    
    horizontal_flip=True,
    
    rotation_range=20,
    
    height_shift_range=0.1,       # No need to shift the image
    width_shift_range=0.1,
    zoom_range=0.1,
    
    shear_range=0.1,              # Seems to be useful
    
    validation_split=VALIDATION_SPLIT,
)

val_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT,
    )

test_generator = ImageDataGenerator(
    rescale=1./255,
)

In [ ]:
train_images = train_generator.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    
    shuffle=True,
    seed=42,
    
    subset='training',
)

val_images = val_generator.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    
    shuffle=True,
    seed=42,
    
    subset='validation'
)


test_images = test_generator.flow_from_directory(
    TEST_PATH,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    color_mode='rgb',
    class_mode='categorical',
    
    batch_size=BATCH_SIZE,
    
    shuffle=False,
    seed=42,
)

NUM_TRAIN_IMAGES = len(train_images.filenames)
NUM_VAL_IMAGES = len(val_images.filenames)
NUM_TEST_IMAGES = len(test_images.filenames)

### LOADING DATA

In [ ]:
from sklearn.preprocessing import LabelEncoder

def give_class_name(y:np.ndarray) -> np.ndarray:
    if len(y.shape) == 2:
        y = y.argmax(axis=1)
    enc = LabelEncoder().fit(CLASS_NAMES)
    y_labeled = enc.inverse_transform(y)
    
    return y_labeled

# display images in a grid function
def display_image_grid(images, labels, pred_val=None,shape=(5, 5), figsize=(10, 10)):
    plt.figure(figsize=figsize)
    m = shape[0] * shape[1]
    
    if images.shape[0] < m:
        raise ValueError("images.shape[0] must equal shape[0] * shape[1]")
    
    if labels.shape[0] != images.shape[0]:
        raise ValueError("labels.shape[0] must equal images.shape[0]")
    
    if pred_val is not None and pred_val.shape[0] != images.shape[0]:
        raise ValueError("pred_val.shape[0] must equal images.shape[0]")
    
    for i in range(m):
        plt.subplot(shape[0], shape[1], i+1)
        plt.imshow(images[i], cmap="gray", interpolation="none")
        title = labels[i]
        if pred_val is not None:
            title = f"{pred_val[i]}"
            if pred_val[i] != labels[i]:
                title += f"\n*({labels[i]})*"
        plt.title(title)
        plt.axis("off")
        # increase vertical space between subplots
        plt.subplots_adjust(wspace=0.1, hspace=0.4)
    plt.show()
    

In [ ]:
num_rows = 8
num_cols = 8

plt.figure(figsize=(20, 15))

images, labels = train_images.next()
for i in range(num_cols * num_rows):
    plt.subplot(num_cols, num_rows, i + 1)
    plt.imshow(images[i])
    plt.title(CLASS_NAMES[labels[i].argmax()])
    plt.axis('off')

plt.show()

## Model

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Rescaling, Input, Add, Activation, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score, log_loss


### Valribles

In [ ]:
INPUT_SHAPE = train_images.next()[0][0].shape
TRAIN_MODELS = True
# TRAIN_MODELS = False


### Useful functions ###

In [ ]:
from tensorflow.keras.metrics import top_k_categorical_accuracy

def predict_label(images, model):
    predictions = model.predict(images)
    return predictions.argmax(axis=1)


# ploting the model training history
def plot_model_performance(history, figsize=(10, 10)):
    preformance = {key: val for key, val in history.history.items() if "loss" not in key}
    losses = {key: val for key, val in history.history.items() if "loss" in key}
    
    plt.figure(figsize=figsize)
    plt.title('Model Performance')
    for key, val in preformance.items():
        plt.plot(val, label=key)
    plt.legend(preformance.keys())
    plt.xlabel('Epoch')
    
    plt.figure(figsize=figsize)
    plt.title('Model Losses')
    for key, val in losses.items():
        plt.plot(val, label=key)
    plt.legend(losses.keys())
    plt.xlabel('Epoch')
    
    plt.show()


# def get_model_performance(y_true, y_pred):
#     if len(y_true.shape) == 2:
#         y_true = y_true.argmax(axis=1)
#     if len(y_pred.shape) == 2:
#         y_pred = y_pred.argmax(axis=1)

#     accuracy = accuracy_score(y_true, y_pred)
#     # top_5_accuracy = top_k_categorical_accuracy(y_true, y_pred, k=5)
#     f1_scr = f1_score(y_true, y_pred, average='weighted')
#     precision_scr = precision_score(y_true, y_pred, average='weighted')
#     recall_scr = recall_score(y_true, y_pred, average='weighted')
    
    
#     performance = {
#         "Accuracy": accuracy,
#         # "top-5-accuracy": top_5_accuracy,
#         "F1 Score": f1_scr,
#         "Precision": precision_scr,
#         "Recall": recall_scr
#     }
    
#     return performance
def compute_performance_metrics(y, y_pred, print_report=True):
    # labels = test_images_.y.argmax(axis=1)
    labels = y
    labels_cat = tf.keras.utils.to_categorical(labels, NUM_CLASSES)
    # pred_cat = model.predict(test_images_)
    pred_cat = y_pred
    pred = pred_cat.argmax(axis=1)

    
    performance_metrics = {}
    performance_metrics["accuracy"] = accuracy_score(labels, pred)
    performance_metrics["top_5_accuracy"] = top_k_categorical_accuracy(labels_cat, pred_cat, k=5).numpy().sum() / len(y)
    performance_metrics["f1_score"] = f1_score(labels, pred, average="macro")
    performance_metrics["precision"] = precision_score(labels, pred, average="macro")
    performance_metrics["recall"] = recall_score(labels, pred, average="macro")
    performance_metrics["loss"] = log_loss(labels_cat, pred_cat)
    
    performance_df.loc[model.name] = performance_metrics
    if print_report:
        return performance_df.loc[model.name]

performance_df = pd.DataFrame(columns=["accuracy", "top_5_accuracy", "precision", "recall", "f1_score", "loss"])


### MODELS

#### MODEL ARCHITECTURE

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import applications

weight_decay = 1e-5
LEARNING_RATE = 4e-4
learning_rate_decay_rate = 0.1

def create_model(model_backbone: str, model_name:str=None, layers=[256]):
    model_name = model_name or model_backbone
    model = Sequential(name=model_name)
    
    model_backbone = get_model_backbone(model_backbone)
    model_backbone.trainable = False
    model.add(model_backbone)
    model.add(GlobalAveragePooling2D())
    
    model.add(Dropout(0.2))
    
    for layer_size in layers:
        model.add(Dense(layer_size, kernel_regularizer=l2(weight_decay)))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(Dropout(0.3))
    
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    return model

def get_model_backbone(model_backbone: str):
    if model_backbone == "VGG16":
        model_backbone = applications.VGG16(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "VGG19":
        model_backbone = applications.VGG19(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "ResNet50V2":
        model_backbone = applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "ResNet101V2":
        model_backbone = applications.ResNet101V2(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "ResNet152V2":
        model_backbone = applications.ResNet152V2(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "InceptionV3":
        model_backbone = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "InceptionResNetV2":
        model_backbone = applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "Xception":
        model_backbone = applications.Xception(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "DenselNet121":
        model_backbone = applications.DenseNet121(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "DenselNet169":
        model_backbone = applications.DenseNet169(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "DenselNet201":
        model_backbone = applications.DenseNet201(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "MobileNet":
        model_backbone = applications.MobileNet(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    elif model_backbone == "MobileNetV2":
        model_backbone = applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
        
    return model_backbone

#### Model Training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger


monitor_metric = 'val_accuracy'
# learning_rate_decay_rate = 0.06
def get_callbacks():
    callbacks = {}
    
    callbacks["EarlyStopping"] = EarlyStopping(
            monitor=monitor_metric,
            patience=6,
            mode = "auto",
            verbose=1,
        )
    
    callbacks["LearningRateScheduler"] = LearningRateScheduler(step_decay)

    callbacks["ModelCheckpoint"] = ModelCheckpoint(
            MODEL_PATH / f"{model.name}.h5",
            monitor=monitor_metric,
            save_best_only=True,
            mode='auto',
            verbose=1,
    )
    
    return callbacks


def step_decay(epoch):
    initial_lr = LEARNING_RATE
    k = learning_rate_decay_rate
    lr = initial_lr * np.exp(-k*epoch)
    return lr



##### VGG16

In [ ]:
backbone_model = "VGG16"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### VGG19

In [ ]:
backbone_model = "VGG19"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### ResNet50V2

In [ ]:
backbone_model = "ResNet50V2"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### ResNet101V2

In [ ]:
backbone_model = "ResNet101V2"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### ResNet152V2

In [ ]:
backbone_model = "ResNet152V2"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### InceptionV3

In [ ]:
backbone_model = "InceptionV3"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### InceptionResNetV2

In [ ]:
backbone_model = "InceptionResNetV2"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### Xception

In [ ]:
backbone_model = "Xception"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### DenseNet121

In [ ]:
backbone_model = "DenseNet121"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### DenseNet169

In [ ]:
backbone_model = "DenseNet169"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### DenseNet201

In [ ]:
backbone_model = "DenseNet201"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### MobileNet

In [ ]:
backbone_model = "MobileNet"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)

##### MobileNetV2

In [ ]:
backbone_model = "MobileNetV2"
model = create_model(backbone_model, layers=[512])
# LEARNING_RATE = 4e-5
# learning_rate_decay_rate = 0.07

callbacks = [callback for callback in get_callbacks().values()]

model.compile(
        optimizer=Adam(
            learning_rate=LEARNING_RATE
            ), 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            'top_k_categorical_accuracy',
            ]
        )
model.summary()

In [ ]:
from tensorflow.keras.models import load_model

train_model = not (os.path.exists(MODEL_PATH / f"{model.name}.h5")) or TRAIN_MODELS

if train_model:
    history = model.fit(train_images, 
                        validation_data=val_images,
                        epochs=50, 
                        callbacks=callbacks,                        
    )
else:
    model_path = MODEL_PATH / f"{model.name}.h5"
    model = load_model(model_path)
    print(f"{model.name} model loaded from {model_path}")
    

In [ ]:
if train_model:
    model.evaluate(test_images)
    plot_model_performance(history)

In [ ]:
test_images.shuffle = False
test_labels = test_images.labels
test_labels_pred_ohe = model.predict(test_images)
test_labels_pred = test_labels_pred_ohe.argmax(axis=1)
compute_performance_metrics(test_labels, test_labels_pred_ohe, True)